chrome://dino/

In [1]:
from __future__ import division

In [2]:
from autopy import *
import gc
from PIL import Image, ImageOps
import numpy as np
import time
import random
from matplotlib import cm
import os.path

In [3]:
import win32gui, win32com.client
import re

class WindowMgr:
    """Encapsulates some calls to the winapi for window management"""
    def __init__ (self):
        """Constructor"""
        self._handle = None

    def find_window(self, class_name, window_name = None):
        """find a window by its class_name"""
        self._handle = win32gui.FindWindow(class_name, window_name)

    def _window_enum_callback(self, hwnd, wildcard):
        '''Pass to win32gui.EnumWindows() to check all the opened windows'''
        if re.match(wildcard, str(win32gui.GetWindowText(hwnd))) != None:
            self._handle = hwnd

    def find_window_wildcard(self, wildcard):
        self._handle = None
        win32gui.EnumWindows(self._window_enum_callback, wildcard)
        
   
    def find_window_position_size(self):
        rect = win32gui.GetWindowRect(self._handle)
        x = rect[0]
        y = rect[1]
        w = rect[2] - x
        h = rect[3] - y
        return (x , y , w , h)
    
    def set_foreground(self):
        """put the window in the foreground"""
        shell = win32com.client.Dispatch("WScript.Shell")
        shell.SendKeys('%')
        win32gui.SetForegroundWindow(self._handle)
        
    def get_screen_image(self):
        x , y , w , h = self.find_window_position_size()
        bmp = bitmap.capture_screen()
        bmp.save('screen.png')
        img = Image.open('screen.png')
        img = img.crop((x + 10 , y + 150  , x+w -10 , y+ h - 200 ))
        img = img.resize((256,256)).convert('L')
        img = ImageOps.invert(img)
        shape = [(0, 55), (w, 50)]
        img = np.asarray(img)
        img = img / 255.0
        img = img.reshape((256,256))
        return img

In [4]:
def is_done(image):
    avg = np.average(image[52:55 , 0:255])
    if avg < 0.05:
        return True 
    else:
        return False

In [5]:
def key_press(code):
        key.toggle(code , True)
        key.toggle(code , False)

In [6]:
def key_tap(code):
    key.tap(code , delay = 0.00000001)
#     key.toggle(code , True)
#     time.sleep(0.000001)
#     key.toggle(code , False)

In [7]:
class Gym:
    def __init__(self):
        self.window = None
        
    def reset(self):
        self.window = WindowMgr()
        self.window.find_window_wildcard(".*chrome://dino/.*")
        self.window.set_foreground()
        key_press(key.Code.F5)
        time.sleep(0.08)
        key_press(key.Code.SPACE)
        img = np.zeros((256,256,3))
        return img
    
    def step(self , action):
        
        
        if action == 2 :
            key_press(key.Code.UP_ARROW)
        if action == 0 :
            key_tap(key.Code.UP_ARROW)
        if action == 3 :
            key_press(key.Code.DOWN_ARROW)
        if action == 1 :
            pass
#         delay = 0 if 0.1 - delay <=0 else 0.1 - delay
#         time.sleep(delay)
        img1 = self.window.get_screen_image()
        time.sleep(1/50)
        img2 = self.window.get_screen_image()
        time.sleep(1/50)
        img3 = self.window.get_screen_image()
        
        img = np.stack([img1 , img2 , img3] , axis=-1)

        done = is_done(img3)


        if done : 
            reward =  +1
        else:
            reward = -100
  
        return img , done , reward

In [8]:
env = Gym()
# env.reset()
# done = True
# while done :
#     observation,done,reward = env.step(np.random.randint(0,3))
# #     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
# #     display(im)

In [9]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow.keras.losses as kloss
import tensorflow.keras.optimizers as opt

In [10]:
class iDQNAgent:
    def __init__(self, state_shape , action_no, memory_size = 1500):
        self.action_no = action_no

        self.gamma = 0.99
        self.epsilon = 1.0

        self.learning_rate = 1e-4
        self.value_c = 0.5
        self.entropy_c = 1e-4
        self.clip_ratio = 0.1
        self.std_adv = True
        self.state_shape = state_shape

        
        self.model = self.build_model(state_shape , action_no)
        self.model.load_weights('a2c.h5')
        self.opt = opt.RMSprop(learning_rate=1e-4) 
        
        ####################################################
    def build_model(self ,input_shape , n_outputs):
        inputs = layers.Input(shape = input_shape , name='state')
        x = layers.Conv2D(64, (4,4) , strides=4 ,padding='same' , kernel_initializer='he_normal')(inputs)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (4,4) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
   
        x = layers.Flatten()(x) 

#         x = layers.Dense(512 , activation='relu')(x)
#         x = layers.Dense(256 , activation='relu')(x)
        
        value = layers.Dense(1 , activation='linear' , name = 'value')(x)
        logits = layers.Dense(n_outputs , activation='linear' , name = 'policy_logits')(x)
        
        model = Model(inputs , [value,logits])
#         model.summary()
        return model
    
        #####################################################
    def act(self , state):
        value, logits = self.model.predict(state)
        action = tf.squeeze(tf.random.categorical(logits, 1), axis=-1)
        return logits, action, np.squeeze(value, axis=-1)
        #####################################################
    def _returns_advantages(self, rewards, dones, values, next_value, standardize_adv):
        returns = np.append(np.zeros_like(rewards), next_value, axis=-1)
 
        # Returns are calculated as discounted sum of future rewards.
        for t in reversed(range(rewards.shape[0])):
            if dones[t] == True: 
                returns[t] = rewards[t] + self.gamma * returns[t + 1] 
            else:
                returns[t] = rewards[t]
        
        returns = returns[:-1]

        # Advantages are equal to returns - baseline (value estimates in our case).
        advantages = returns - values
        
        if standardize_adv:
            advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-10)

        return returns, advantages
        ##########################################################
    
    def _logits_loss_a2c(self, actions, advantages, logits):

        # Sparse categorical CE loss obj that supports sample_weight arg on `call()`.
        # `from_logits` argument ensures transformation into normalized probabilities.
        weighted_sparse_ce = kloss.SparseCategoricalCrossentropy(from_logits=True)

        # Policy loss is defined by policy gradients, weighted by advantages.
        # Note: we only calculate the loss on the actions we've actually taken.
        actions = tf.cast(actions, tf.int32)
        policy_loss = weighted_sparse_ce(actions, logits, sample_weight=advantages)

        # Entropy loss can be calculated as cross-entropy over itself.
        probs = tf.nn.softmax(logits)
        entropy_loss = kloss.categorical_crossentropy(probs, probs)

        # We want to minimize policy and maximize entropy losses.
        # Here signs are flipped because the optimizer minimizes.
        return policy_loss - self.entropy_c * entropy_loss
        #############################################################
    def _value_loss(self, returns, value):
        # Value loss is typically MSE between value estimates and returns.
        return self.value_c * kloss.mean_squared_error(returns, value)


In [11]:
episode_count = 1000000
time_steps =    100
batch_size = 256

In [12]:
agent = iDQNAgent((256 , 256 , 3) ,4)

In [13]:
actions = np.empty((batch_size), dtype=np.int32)
rewards, dones, values = np.empty((3, batch_size))
states = np.empty((batch_size,) + agent.state_shape)
old_logits = np.empty((batch_size, agent.action_no), dtype=np.float32)

ep_rewards = [0.0]
next_obs = env.reset()

for episode in range(episode_count):

    for step in range(batch_size) :
        states[step] = next_obs.copy()
        old_logits[step], actions[step], values[step] = agent.act(next_obs[None, :])
#         print('action',actions[step] )
        next_obs, dones[step] , rewards[step] = env.step(actions[step])
        ep_rewards[-1] += rewards[step]

        if dones[step] == False:
            ep_rewards.append(0.0)
            next_obs = env.reset()
            print('Game number: {} Rewad : {}'.format(len(ep_rewards) - 1,round(ep_rewards[-2], 2)))
            
    _, _, next_value = agent.act(next_obs[None, :])

    returns, advs = agent._returns_advantages(rewards, dones, values, next_value, agent.std_adv)
    
    with tf.GradientTape() as tape:
        v, logits = agent.model(states, training=True)

        logit_loss = agent._logits_loss_a2c(actions, advs, logits)

        value_loss = agent._value_loss(returns, v)
        loss = logit_loss + value_loss
#         print("loss: {} logit_loss: {} value_loss: {}".format(loss,logit_loss,value_loss))
    grads = tape.gradient(loss, agent.model.trainable_variables)
    agent.opt.apply_gradients(zip(grads, agent.model.trainable_variables))
    print('policy loss: {}, value loss :{}'.format(np.mean(loss),np.mean(value_loss) ))
    if episode % 1000 == 0:
        agent.model.save_weights('a2c.h5')

Game number: 1 Rewad : -52.0
Game number: 2 Rewad : -68.0
Game number: 3 Rewad : -82.0
Game number: 4 Rewad : -56.0
Game number: 5 Rewad : -73.0
Game number: 6 Rewad : -79.0
Game number: 7 Rewad : -78.0
Game number: 8 Rewad : -69.0
policy loss: 228.67874145507812, value loss :228.78860473632812
Game number: 9 Rewad : -95.0
Game number: 10 Rewad : -84.0
Game number: 11 Rewad : -79.0
Game number: 12 Rewad : -86.0
Game number: 13 Rewad : -86.0
Game number: 14 Rewad : -75.0
Game number: 15 Rewad : -63.0
Game number: 16 Rewad : -75.0
Game number: 17 Rewad : -86.0
Game number: 18 Rewad : -82.0
Game number: 19 Rewad : -43.0
policy loss: 216.65904235839844, value loss :216.70166015625
Game number: 20 Rewad : -96.0
Game number: 21 Rewad : -65.0
Game number: 22 Rewad : -84.0
Game number: 23 Rewad : -86.0
Game number: 24 Rewad : -42.0
Game number: 25 Rewad : -72.0
Game number: 26 Rewad : -84.0
Game number: 27 Rewad : -83.0
Game number: 28 Rewad : -62.0
Game number: 29 Rewad : -76.0
policy loss: 2

policy loss: 138.7354736328125, value loss :138.69021606445312
Game number: 216 Rewad : -89.0
Game number: 217 Rewad : -66.0
Game number: 218 Rewad : -46.0
Game number: 219 Rewad : -86.0
Game number: 220 Rewad : -75.0
Game number: 221 Rewad : -72.0
Game number: 222 Rewad : -56.0
Game number: 223 Rewad : -61.0
policy loss: 220.24371337890625, value loss :220.21495056152344
Game number: 224 Rewad : -90.0
Game number: 225 Rewad : -68.0
Game number: 226 Rewad : -85.0
Game number: 227 Rewad : -55.0
Game number: 228 Rewad : -12.0
Game number: 229 Rewad : -82.0
Game number: 230 Rewad : -86.0
Game number: 231 Rewad : -72.0
policy loss: 528.1762084960938, value loss :528.166015625
Game number: 232 Rewad : -92.0
Game number: 233 Rewad : -67.0
Game number: 234 Rewad : -66.0
Game number: 235 Rewad : -54.0
Game number: 236 Rewad : -82.0
Game number: 237 Rewad : -86.0
Game number: 238 Rewad : -74.0
Game number: 239 Rewad : -69.0
Game number: 240 Rewad : -86.0
Game number: 241 Rewad : -82.0
policy lo

Game number: 428 Rewad : -79.0
Game number: 429 Rewad : -69.0
Game number: 430 Rewad : -85.0
Game number: 431 Rewad : -83.0
Game number: 432 Rewad : -68.0
policy loss: 230.16908264160156, value loss :230.16839599609375
Game number: 433 Rewad : -90.0
Game number: 434 Rewad : -85.0
Game number: 435 Rewad : -86.0
Game number: 436 Rewad : -67.0
Game number: 437 Rewad : -67.0
Game number: 438 Rewad : -84.0
Game number: 439 Rewad : -68.0
Game number: 440 Rewad : -72.0
Game number: 441 Rewad : -52.0
policy loss: 195.74520874023438, value loss :195.76364135742188
Game number: 442 Rewad : -60.0
Game number: 443 Rewad : -72.0
Game number: 444 Rewad : -72.0
Game number: 445 Rewad : -60.0
Game number: 446 Rewad : -82.0
Game number: 447 Rewad : -80.0
Game number: 448 Rewad : -73.0
Game number: 449 Rewad : -72.0
Game number: 450 Rewad : -67.0
policy loss: 133.1977996826172, value loss :133.22103881835938
Game number: 451 Rewad : -87.0
Game number: 452 Rewad : -72.0
Game number: 453 Rewad : -81.0
Gam

Game number: 641 Rewad : -57.0
Game number: 642 Rewad : -49.0
Game number: 643 Rewad : -61.0
Game number: 644 Rewad : -85.0
Game number: 645 Rewad : -79.0
Game number: 646 Rewad : -86.0
Game number: 647 Rewad : -85.0
Game number: 648 Rewad : -78.0
Game number: 649 Rewad : -60.0
policy loss: 201.40267944335938, value loss :201.47171020507812
Game number: 650 Rewad : -88.0
Game number: 651 Rewad : -85.0
Game number: 652 Rewad : -58.0
Game number: 653 Rewad : -56.0
Game number: 654 Rewad : -40.0
Game number: 655 Rewad : -81.0
Game number: 656 Rewad : -76.0
Game number: 657 Rewad : -84.0
policy loss: 279.878173828125, value loss :279.8843994140625
Game number: 658 Rewad : -72.0
Game number: 659 Rewad : -86.0
Game number: 660 Rewad : -86.0
Game number: 661 Rewad : -67.0
Game number: 662 Rewad : -73.0
Game number: 663 Rewad : -73.0
Game number: 664 Rewad : -62.0
Game number: 665 Rewad : -44.0
policy loss: 271.7950439453125, value loss :271.80596923828125
Game number: 666 Rewad : -61.0
Game n

Game number: 853 Rewad : -73.0
Game number: 854 Rewad : -61.0
Game number: 855 Rewad : -59.0
Game number: 856 Rewad : -71.0
Game number: 857 Rewad : -66.0
Game number: 858 Rewad : -76.0
Game number: 859 Rewad : -82.0
Game number: 860 Rewad : -85.0
policy loss: 162.129150390625, value loss :162.12344360351562
Game number: 861 Rewad : -65.0
Game number: 862 Rewad : -84.0
Game number: 863 Rewad : -79.0
Game number: 864 Rewad : -67.0
Game number: 865 Rewad : -82.0
Game number: 866 Rewad : -75.0
Game number: 867 Rewad : -77.0
Game number: 868 Rewad : -48.0
Game number: 869 Rewad : -68.0
policy loss: 183.62075805664062, value loss :183.59609985351562
Game number: 870 Rewad : -88.0
Game number: 871 Rewad : -80.0
Game number: 872 Rewad : -84.0
Game number: 873 Rewad : -86.0
Game number: 874 Rewad : -65.0
Game number: 875 Rewad : -72.0
Game number: 876 Rewad : -85.0
Game number: 877 Rewad : -84.0
Game number: 878 Rewad : -85.0
Game number: 879 Rewad : -67.0
Game number: 880 Rewad : -64.0
policy

...

Game number: 9869 Rewad : -49.0
Game number: 9870 Rewad : -72.0
Game number: 9871 Rewad : -72.0
Game number: 9872 Rewad : -78.0
Game number: 9873 Rewad : -47.0
Game number: 9874 Rewad : -66.0
Game number: 9875 Rewad : -84.0
Game number: 9876 Rewad : -39.0
policy loss: 283.8788146972656, value loss :283.97314453125
Game number: 9877 Rewad : -94.0
Game number: 9878 Rewad : -80.0
Game number: 9879 Rewad : -64.0
Game number: 9880 Rewad : -82.0
Game number: 9881 Rewad : -64.0
Game number: 9882 Rewad : -79.0
Game number: 9883 Rewad : -55.0
Game number: 9884 Rewad : -87.0
Game number: 9885 Rewad : -50.0
policy loss: 211.2947998046875, value loss :211.3896484375
Game number: 9886 Rewad : -92.0
Game number: 9887 Rewad : -66.0
Game number: 9888 Rewad : -71.0
Game number: 9889 Rewad : -73.0
Game number: 9890 Rewad : -48.0
Game number: 9891 Rewad : -33.0
Game number: 9892 Rewad : -78.0
policy loss: 306.068115234375, value loss :306.08367919921875
Game number: 9893 Rewad : -29.0
Game number: 9894 R

Game number: 10071 Rewad : -55.0
Game number: 10072 Rewad : -63.0
Game number: 10073 Rewad : -72.0
Game number: 10074 Rewad : -69.0
Game number: 10075 Rewad : -54.0
Game number: 10076 Rewad : -47.0
policy loss: 224.70443725585938, value loss :224.81387329101562
Game number: 10077 Rewad : -91.0
Game number: 10078 Rewad : -82.0
Game number: 10079 Rewad : -86.0
Game number: 10080 Rewad : -72.0
Game number: 10081 Rewad : -86.0
Game number: 10082 Rewad : -75.0
Game number: 10083 Rewad : -76.0
Game number: 10084 Rewad : -58.0
Game number: 10085 Rewad : -60.0
Game number: 10086 Rewad : -77.0
policy loss: 161.61849975585938, value loss :161.63125610351562
Game number: 10087 Rewad : -65.0
Game number: 10088 Rewad : -44.0
Game number: 10089 Rewad : -64.0
Game number: 10090 Rewad : -73.0
Game number: 10091 Rewad : -73.0
Game number: 10092 Rewad : -72.0
Game number: 10093 Rewad : -57.0
policy loss: 219.36709594726562, value loss :219.39263916015625
Game number: 10094 Rewad : -85.0
Game number: 100

Game number: 10269 Rewad : -86.0
Game number: 10270 Rewad : -69.0
Game number: 10271 Rewad : -46.0
Game number: 10272 Rewad : -77.0
Game number: 10273 Rewad : -72.0
policy loss: 200.33303833007812, value loss :200.35711669921875
Game number: 10274 Rewad : -86.0
Game number: 10275 Rewad : -64.0
Game number: 10276 Rewad : -73.0
Game number: 10277 Rewad : -72.0
Game number: 10278 Rewad : -55.0
Game number: 10279 Rewad : -45.0
Game number: 10280 Rewad : -81.0
Game number: 10281 Rewad : -69.0
policy loss: 219.73562622070312, value loss :219.79861450195312
Game number: 10282 Rewad : -93.0
Game number: 10283 Rewad : -86.0
Game number: 10284 Rewad : -39.0
Game number: 10285 Rewad : -66.0
Game number: 10286 Rewad : -63.0
Game number: 10287 Rewad : -85.0
Game number: 10288 Rewad : -82.0
Game number: 10289 Rewad : -67.0
Game number: 10290 Rewad : -72.0
policy loss: 246.74745178222656, value loss :246.75396728515625
Game number: 10291 Rewad : -93.0
Game number: 10292 Rewad : -64.0
Game number: 102

Game number: 10469 Rewad : -93.0
Game number: 10470 Rewad : -39.0
Game number: 10471 Rewad : -74.0
Game number: 10472 Rewad : -72.0
Game number: 10473 Rewad : -54.0
Game number: 10474 Rewad : -72.0
Game number: 10475 Rewad : -86.0
policy loss: 357.21966552734375, value loss :357.30682373046875
Game number: 10476 Rewad : -54.0
Game number: 10477 Rewad : -41.0
Game number: 10478 Rewad : -82.0
Game number: 10479 Rewad : -61.0
Game number: 10480 Rewad : -80.0
Game number: 10481 Rewad : -56.0
Game number: 10482 Rewad : -66.0
Game number: 10483 Rewad : -70.0
policy loss: 250.04306030273438, value loss :250.16162109375
Game number: 10484 Rewad : -96.0
Game number: 10485 Rewad : -62.0
Game number: 10486 Rewad : -82.0
Game number: 10487 Rewad : -35.0
Game number: 10488 Rewad : -82.0
Game number: 10489 Rewad : -84.0
Game number: 10490 Rewad : -71.0
Game number: 10491 Rewad : -52.0
policy loss: 298.2952575683594, value loss :298.3857421875
Game number: 10492 Rewad : -84.0
Game number: 10493 Rewad

KeyboardInterrupt: 

In [ ]:
# state = env.reset()
# done = True
# while done :
#     q_values = agent.q_model.predict(np.expand_dims(state , axis=0))
#     action = np.argmax(q_values[0])
#     next_state , done ,reward  = env.step(action ,0)
#     state = next_state
#     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
#     display(im)